In [1]:
import pandas as pd
import os
import numpy as np
import pandas_profiling

# This line is needed to display plots inline in Jupyter Notebook
%matplotlib inline

# Required for basic python plotting functionality
import matplotlib.pyplot as plt

# Required for formatting dates later in the case
import datetime
import matplotlib.dates as mdates

# Required to display image inline
from IPython.display import Image

# Advanced plotting functionality with seaborn
import seaborn as sns
sns.set(style="whitegrid") # can set style depending on how you'd like it to look

import folium  #needed for interactive map
from folium.plugins import HeatMap

import geopandas
from shapely.geometry import Polygon
from shapely.geometry import Point
from geopandas import GeoDataFrame

plotsize = (15,6)

In [2]:
path = 'Dataset/Parquets'
yellow_trips = pd.read_parquet(path + '/yellow_trips_filtered.parquet', engine = 'pyarrow')

weather = pd.read_parquet(path + '/weather_by_day.parquet', engine = 'pyarrow')

In [14]:
yellow_trips['pickup_datetime_date'] = yellow_trips['pickup_datetime'].dt.to_period('d')
yellow_trips['pickup_datetime_date'] = yellow_trips['pickup_datetime_date'].values.astype('datetime64[D]')
#pd.to_datetime(yellow_trips['pickup_datetime'].dt.strftime('%d/%m/%Y'), format = '%d/%m/%y')
yellow_trips.head()

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount,pickup_month,dropoff_month,amount_per_distance,pickup_datetime_date
0,2014-04-03 18:28:10,2014-04-03 18:28:10,-74.006033,40.706284,-73.918837,40.744946,1,8.70,34.8,201404,201404,4.000000,2014-04-03
1,2014-04-16 15:42:00,2014-04-16 15:42:00,-73.979558,40.749357,0.000000,0.000000,1,9.37,30.0,201404,201404,3.201708,2014-04-16
2,2014-04-13 18:04:00,2014-04-13 18:04:00,-73.956453,40.775307,-73.954792,40.784992,1,0.89,6.5,201404,201404,7.303371,2014-04-13
3,2014-05-21 19:33:00,2014-05-21 19:33:00,-73.987212,40.757850,-73.960198,40.775472,1,2.40,12.5,201405,201405,5.208333,2014-05-21
4,2014-05-30 16:28:00,2014-05-30 16:28:00,-73.974292,40.755397,-74.011867,40.704222,3,5.58,24.5,201405,201405,4.390681,2014-05-30


In [5]:
weather.head()

,date,max_temp,min_temp,avg_temp,precipitation,snowfall,snow_depth,location,latitude,longitude,index_right,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code
0,2014-01-01,34,24,29.0,0.00,0.000000,0.0,JFK,40.64749,-73.779982,188,QN98,228199205.918,081,Airport,136232.037706,Queens,4
1,2014-01-02,33,19,26.0,0.13,2.400000,0.0,JFK,40.64749,-73.779982,188,QN98,228199205.918,081,Airport,136232.037706,Queens,4
2,2014-01-03,19,9,14.0,0.22,5.500000,7.0,JFK,40.64749,-73.779982,188,QN98,228199205.918,081,Airport,136232.037706,Queens,4
3,2014-01-04,31,3,17.0,0.00,0.000000,7.0,JFK,40.64749,-73.779982,188,QN98,228199205.918,081,Airport,136232.037706,Queens,4
4,2014-01-05,49,23,36.0,0.10,0.855556,6.0,JFK,40.64749,-73.779982,188,QN98,228199205.918,081,Airport,136232.037706,Queens,4


In [9]:
yellow_trips.dtypes

pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
pickup_longitude               float64
pickup_latitude                float64
dropoff_longitude              float64
dropoff_latitude               float64
passenger_count                  int64
trip_distance                  float64
total_amount                   float64
pickup_month                     int64
dropoff_month                    int64
amount_per_distance            float64
pickup_datetime_date            object
dtype: object

In [ ]:
yellow_trips_weather = pd.concat([yellow_trips, weather], keys=['pickup_datetime_date', 'date'],)

In [8]:
print(yellow_trips.shape)
yellow_trips_weather = pd.merge_asof(yellow_trips, weather, direction='nearest',
                                          left_on = 'pickup_datetime_date',
                                          right_on = 'date')
print(yellow_trips_weather.shape)

(7588170, 13)


MergeError: incompatible merge keys [0] dtype('O') and dtype('<M8[ns]'), must be the same type